<a href="https://colab.research.google.com/github/roitraining/SparkforDataEngineers/blob/Development/Ch06_ClassificationAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sys
sys.path.append('/home/student/ROI/SparkProgram')

import pandas as pd
import numpy

import matplotlib as mp
from matplotlib import pyplot as plt
#from IPython.display import display

import pyspark_helpers as pyh
sc, spark, conf = pyh.initspark()
from pyspark_helpers import display

initializing pyspark
pyspark initialized


In [3]:
filename = 'bank.csv'
df = spark.read.csv(f'/home/student/ROI/Spark/datasets/finance/{filename}', header = True, inferSchema = True)
display(df)

# Save a pointer to the raw data
dfRawFile = df


AnalysisException: 'Path does not exist: file:/home/student/ROI/Spark/datasets/finance/bank.csv;'

In [0]:
# Let's just keep a few fields to start with for simplicity
numeric_features = ['age','balance', 'duration', 'pdays']
categorical_features = ['job', 'marital', 'education', 'housing', 'loan', 'contact', 'campaign', 'poutcome', 'deposit']

# numeric_features = ['balance', 'duration', 'age']
# categorical_features = ['marital', 'education']
target_label = 'default'


df = dfRawFile.select(numeric_features + categorical_features + [target_label])
display(df)
print(df.take(1))

### Explore numeric features.

In [0]:
%matplotlib inline
# import imp
# imp.reload(pyh)

pyh.describe_numeric_features(df, numeric_features)
pyh.scatter_matrix(df, numeric_features)


In [0]:
%matplotlib inline
# print('Convert string features to indexed numeric values')
# df1 = pyh.StringIndexEncode(df, categorical_features + [target_label])
# display(df1)

# print('Convert indexed features to OneHotEncoded sparse matrix')
# df2 = pyh.OneHotEncode(df1, categorical_features)
# display(df2)

# print('Assemble all features and target label into vectors')
# df3 = pyh.AssembleFeatures(df2, categorical_features, numeric_features, target_label + '_Index')
# display(df3)
# df3.printSchema()

dfML = pyh.MakeMLDataFrame(df, categorical_features, numeric_features, target_label)
display(dfML)
dfML.printSchema()
labelCnt = dfML.groupBy('label').count()
display(labelCnt)



In [0]:
labelCnt.toPandas().plot(kind = 'bar')


In [0]:
# dfML.write.format('libsvm').save('testsave')
# dfML = spark.read.format('libsvm').load('testsave')
# x.printSchema()
# display(x)

In [0]:
train, test = dfML.randomSplit([.7,.3], seed = 1000)
print (f'Training set row count {train.count()}')
print (f'Testing set row count {test.count()}')
      

In [0]:
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'label', maxDepth = 3)
dtModel = dt.fit(train)
print('DT Trained')

filename1 = filename.replace('.','_') + '_DT_trainedModel'
dtModel.write().overwrite().save(filename1)
print('DT Saved')


In [0]:
dtPredictions, dtLog = pyh.predict_and_evaluate(dtModel, test)


In [0]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=10)
lrModel = lr.fit(train)
print('LR Trained')

filename1 = filename.replace('.','_') + '_LR_trainedModel'
lrModel.write().overwrite().save(filename1)
print('LR Saved')


In [0]:
#lrModel2 = LogisticRegression.load(filename1)

In [0]:
lrPredictions, lrLog = pyh.predict_and_evaluate(lrModel, test)

In [0]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'label')
rfModel = rf.fit(train)
print('RF Trained')

filename1 = filename.replace('.','_') + '_RF_trainedModel'
rfModel.write().overwrite().save(filename1)
print('RF Saved')



In [0]:
rfPredictions, rfLog = pyh.predict_and_evaluate(rfModel, test)



In [0]:
from pyspark.ml.classification import GBTClassifier
gbt = GBTClassifier(maxIter=10)
gbtModel = gbt.fit(train)
print ('GBT Trained')

filename1 = filename.replace('.','_') + '_GBT_trainedModel'
rfModel.write().overwrite().save(filename1)
print ('GBT Saved')



In [0]:
gbtPredictions, gbtLog = pyh.predict_and_evaluate(gbtModel, test)



In [0]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# specify layers for the neural network:
# input layer of size 13 (features), two intermediate of size 5 and 4
# and output of size 2 (classes)
layers = [13, 5, 4, 2]

nn = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)
nnModel = nn.fit(train)
print ('NN Trained')

filename1 = filename.replace('.','_') + '_NN_trainedModel'
nnModel.write().overwrite().save(filename1)
print ('NN Saved')



In [0]:
nnPredictions = nnModel.transform(test)
print(nnPredictions)
nnPredictions.printSchema()
print (nnPredictions.count())
nnPredictions.take(1)
# predictionAndLabels = nnPredictions.select("prediction", "label")
# #display(predictionAndLabels)
# print(predictionAndLabels.collect())
# evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
# print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))